# Splitting data

Splitting data is a crucial step when it comes to validating models, let's demonstrate a couple ways to do just that. First, we load data.

In [1]:
import pandas as pd
import spacy
import umap
import numpy as np 
from pathlib import Path
import sys
sys.path.append("..")

from ml_editor.data_processing import format_raw_df

data_path = Path('../data/writers.csv')
df = pd.read_csv(data_path)
df = format_raw_df(df.copy())

/Users/emmanuel.ameisen/ml_editor/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Random Split

The simplest way to generate a test set is to randomly split data between train and test.

In [2]:
from ml_editor.data_processing import get_random_train_test_split, get_vectorized_inputs_and_label

train_df_rand, test_df_rand = get_random_train_test_split(df[df["is_question"]], test_size=0.3, random_state=40)

In [3]:
print("%s questions in training, %s in test." % (len(train_df_rand),len(test_df_rand)))
train_owners = set(train_df_rand['OwnerUserId'].values)
test_owners = set(test_df_rand['OwnerUserId'].values)
print("%s different owners in the training set" % len(train_df_rand))
print("%s different owners in the testing set" % len(test_df_rand))
print("%s owners appear in both sets" % len(train_owners.intersection(test_owners)))

5579 questions in training, 2392 in test.
5579 different owners in the training set
2392 different owners in the testing set
596 owners appear in both sets


## Author Split

Some authors may be more successful on average, and that may due to factors other than the quality of their formulation such as their popularity. Ideally, we would want to make sure that a given author only appears in either the training set or the validation set. This guarantee that a model will not be able to leverage information to identify a given author and use it to predict more easily.

To remove this potential source of bias, let's split data by author

In [4]:
from ml_editor.data_processing import get_split_by_author

train_author, test_author = get_split_by_author(df[df["is_question"]], test_size=0.3, random_state=40)

print("%s questions in training, %s in test." % (len(train_author),len(test_author)))
train_owners = set(train_author['OwnerUserId'].values)
test_owners = set(test_author['OwnerUserId'].values)
print("%s different owners in the training set" % len(train_owners))
print("%s different owners in the testing set" % len(test_owners))
print("%s owners appear in both sets" % len(train_owners.intersection(test_owners)))

5676 questions in training, 2295 in test.
2723 different owners in the training set
1167 different owners in the testing set
0 owners appear in both sets


And now, we have a better data split that we will use going forward!